In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.move('/content/model_epoch_1/model.bin', '/content/drive/My Drive')

Error: Destination path '/content/drive/My Drive/model.bin' already exists

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Load your dataset
df = pd.read_csv('df_file.csv')

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the split datasets to CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)


In [ ]:
# Define the maximum sequence length
max_seq_length = 512

# Tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer):
        self.data = pd.read_csv(file_path)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data.iloc[idx]['Text']
        label = self.data.iloc[idx]['Label']
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        return {'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)}

In [ ]:
train_dataset = MyDataset(file_path='train.csv', tokenizer=tokenizer)  # Pass the tokenizer here
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
import os

In [ ]:
from transformers import AdamW
import os

# Define training parameters
batch_size = 8
lr = 5e-5
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=lr)
train_dataset = MyDataset('train.csv', tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item()}")
    # Save the model and tokenizer after each epoch
    output_dir = f"model_epoch_{epoch+1}"
    os.makedirs(output_dir, exist_ok=True)
    torch.save(model.state_dict(), 'model_epoch_1/model.bin')
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3 - Loss: 0.15260545909404755
Epoch 2/3 - Loss: 0.005587780848145485
Epoch 3/3 - Loss: 0.004966469015926123


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd

# Sample tweet
tweet = "I love natural language processing. Topic modeling is interesting."

# Preprocess the tweet
tweets = [tweet]

# Vectorize the tweet
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(tweets)

# Fit LDA model
num_topics = 2  # Number of topics to identify
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Get the topics for the tweet
tweet_topics = lda.transform(X)

# Display the topics with their most relevant words
feature_names = vectorizer.get_feature_names_out()
topic_names = [f"Topic {i+1}: " + " ".join([feature_names[idx] for idx in topic.argsort()[:-5:-1]]) for i, topic in enumerate(lda.components_)]

df = pd.DataFrame(tweet_topics, columns=topic_names)
print(df)


   Topic 1: chadwick stage progressed diagnosed  \
0                                      0.967523   

   Topic 2: stage chadwick iii years  
0                           0.032477  


In [ ]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, BertTokenizerFast

# Load the pretrained model with its configuration
model_name_or_path = 'bert-base-uncased'
config = BertConfig.from_pretrained(model_name_or_path)
model = BertForSequenceClassification.from_pretrained(model_name_or_path, config=config)

# Load the custom tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Load other custom files (e.g., special_tokens_map.json, tokenizer_config.json, vocab.txt, model.safetensors)
# You may need to adjust the paths based on your actual file locations
special_tokens_map = tokenizer.load_special_tokens_map('model_epoch_7/special_tokens_map.json')
tokenizer_config = tokenizer.load_tokenizer_config('model_epoch_7/tokenizer_config.json')
vocab = tokenizer.load_vocab('model_epoch_7/vocab.txt')
model_safetensors = torch.load('model_epoch_7/model.safetensors')

# Apply the custom tokenizer to the model
model.resize_token_embeddings(len(tokenizer))

# Set the model's configuration to the custom configuration
model.config = config

# Save the fine-tuned model
output_dir = "fine_tuned_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'BertTokenizerFast' object has no attribute 'load_special_tokens_map'

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('model_epoch_7/')

# Load the model
model = BertForSequenceClassification.from_pretrained('model_epoch_7/')

# Set the model to evaluation mode
model.eval()

# Define the mapping from numeric labels to topic names
label_to_topic = {
    0: 'Politics',
    1: 'Sport',
    2: 'Technology',
    3: 'Entertainment',
    4: 'Business'
}

def classify_tweet(tweet, model, tokenizer):
    # Tokenize the input tweet
    inputs = tokenizer(tweet, return_tensors='pt')

    # Classify the tweet using the model
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label = logits.argmax().item()

    # Map the predicted label to the topic name
    predicted_topic = label_to_topic[predicted_label]

    return predicted_topic


# Example usage
tweet = "I love technology"
predicted_label = classify_tweet(tweet, model, tokenizer)
print(f"Predicted label: {predicted_label}")


Predicted label: Technology


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


## webapp

In [ ]:
import streamlit as st
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('model_epoch_7/')

# Load the model
model = BertForSequenceClassification.from_pretrained('model_epoch_7/')

# Set the model to evaluation mode
model.eval()

# Define the mapping from numeric labels to topic names
label_to_topic = {
    0: 'Politics',
    1: 'Sport',
    2: 'Technology',
    3: 'Entertainment',
    4: 'Business'
}

In [ ]:
# Title of the app
st.title('Tweet Topic Classifier')

# Text input box for user input
user_input = st.text_area('Enter your tweet here:', '')

# Button to classify the tweet
if st.button('Classify'):
    # Classify the tweet using the BERT model
    predicted_topic = classify_tweet(user_input, model, tokenizer)

    # Display the predicted topic
    st.write(f'Predicted Topic: {predicted_topic}')

2024-03-20 01:26:12.790 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.178.52:8501

  Stopping...
^C


#Twitter

In [ ]:
# Load your Twitter dataset
df = pd.read_csv('twitter_dataset.csv')

# Use only the 'Text' column for topic modeling
texts = df['Text'].tolist()

In [ ]:
# Tokenize the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(text) for text in texts]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate masked input sequences for BERT Masked Language Modeling (MLM)
mlm_input_texts = ["[CLS] " + " ".join(tokens) + " [SEP]" for tokens in tokenized_texts]
mlm_input_ids = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)) for text in mlm_input_texts]

# Fine-tune BERT for Masked Language Modeling
model = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device)  # Move model to GPU
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.train()
for epoch in range(1):  # Adjust the number of epochs as needed
    for input_ids in mlm_input_ids:
        input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Move input_ids to GPU
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Save model checkpoint
    checkpoint_path = f"bert_checkpoint_epoch_{epoch}.pt"
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch
from transformers import BertForMaskedLM

# Create a new instance of the BertForMaskedLM model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Load the saved model checkpoint
checkpoint_path = "bert_checkpoint_epoch_0.pt"
model_checkpoint = torch.load(checkpoint_path)

# Extract the 'model_state_dict' from the checkpoint
model_state_dict = model_checkpoint['model_state_dict']

# Load the 'model_state_dict' into the model
model.load_state_dict(model_state_dict)

# Make sure to move the model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

# Fine-tune the model
model.train()
for epoch in range(1):  # Example: 3 epochs
    for input_ids in mlm_input_ids:
        input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Move input_ids to GPU
        outputs = model(input_ids=input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Save the retrained model
model.save_pretrained('retrained_bert_model')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer

# Create and save the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.save_pretrained("retrained_bert_model")

('retrained_bert_model/tokenizer_config.json',
 'retrained_bert_model/special_tokens_map.json',
 'retrained_bert_model/vocab.txt',
 'retrained_bert_model/added_tokens.json')

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example input text
input_text = "I love natural language processing"
encoded_input = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True)

# Generate embeddings
with torch.no_grad():
    outputs = model(**encoded_input)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()

# Example topics (assuming 2 topics for illustration)
topics = ['topic1', 'topic2']
# Example topic labels for training (0 for topic1, 1 for topic2)
labels = [0, 1]

# Train a logistic regression classifier
clf = LogisticRegression(random_state=42)
clf.fit(embeddings.numpy(), labels)

# Predict topic for new input
new_input_text = "I enjoy learning about machine learning"
encoded_new_input = tokenizer(new_input_text, return_tensors='pt', padding=True, truncation=True)

with torch.no_grad():
    outputs = model(**encoded_new_input)
    new_embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()

predicted_topic_index = clf.predict(new_embeddings.numpy())[0]
predicted_topic = topics[predicted_topic_index]
print(f"Predicted Topic: {predicted_topic}")


ValueError: Expected 2D array, got 1D array instead:
array=[ 2.08781347e-01  3.45988959e-01 -2.03055382e-01 -1.22337632e-01
  1.31165445e-01 -1.15216710e-01  6.47712499e-02  7.56084800e-01
 -1.70435041e-01 -2.39162102e-01 -2.15514854e-01 -1.25003323e-01
  2.46141050e-02  9.96536165e-02 -1.06602862e-01  4.93316613e-02
  1.79790601e-01  1.42797500e-01  1.49333971e-02  4.80228186e-01
  1.39139816e-01  1.19294882e-01 -1.58279110e-02  2.94743478e-01
  5.27688563e-01 -1.48743168e-01 -2.08438978e-01  2.22036824e-01
 -2.15714440e-01 -2.33359560e-01  7.06152385e-03 -1.81175470e-02
 -7.00873369e-03  2.62271613e-01 -1.77604571e-01 -3.63791674e-01
 -2.57358819e-01 -1.88217670e-01 -3.54711235e-01  1.02463357e-01
 -1.29411304e-02 -5.82791828e-02 -1.34163499e-01 -2.63754100e-01
 -2.29441258e-03 -1.62283018e-01 -2.76572794e-01  2.43531793e-01
  5.14717996e-01  7.06096068e-02 -1.93929046e-01  3.35880220e-01
  1.40120044e-01  1.47341520e-01 -8.31267610e-02  5.41195750e-01
 -2.36949891e-01 -2.81938016e-0

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Flatten embeddings
flat_embeddings = embeddings.squeeze().numpy()

# Perform K-means clustering
num_clusters = 5  # Number of clusters/topics
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(flat_embeddings)

# Map words to clusters
word_clusters = {}
for i, (word, _) in enumerate(word_embeddings):
    cluster = cluster_labels[i]
    if cluster not in word_clusters:
        word_clusters[cluster] = []
    word_clusters[cluster].append(word)

# Print clusters
for cluster, words in word_clusters.items():
    print(f"Cluster {cluster}: {', '.join(words)}")


Cluster 3: [CLS]
Cluster 2: i, love
Cluster 1: natural, language
Cluster 4: processing
Cluster 0: [SEP]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# Load the fine-tuned BERT model and tokenizer
model_path = "retrained_bert_model"
model = BertModel.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Define the tweet
tweet = "It is with immeasurable grief that we confirm the passing of Chadwick Boseman. Chadwick was diagnosed with stage III colon cancer in 2016, and battled with it these last 4 years as it progressed to stage"

# Tokenize the tweet
tokenized_tweet = tokenizer.tokenize(tweet)

# Add special tokens [CLS] and [SEP]
tokenized_tweet = ["[CLS]"] + tokenized_tweet + ["[SEP]"]

# Convert tokens to IDs
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_tweet)

# Create a tensor for the input tokens
tokens_tensor = torch.tensor([indexed_tokens])

# Perform inference
model.eval()
with torch.no_grad():
    outputs = model(tokens_tensor)
    hidden_states = outputs.last_hidden_state

# Extract contextual embeddings (hidden states) for each token
contextual_embeddings = hidden_states.squeeze(0)  # Remove the batch dimension

# Convert token IDs to tokens
decoded_tokens = tokenizer.convert_ids_to_tokens(indexed_tokens)

# Print the tokens and their corresponding contextual embeddings
for i, token in enumerate(decoded_tokens):
    print(f"Token: {token}, Contextual Embedding: {contextual_embeddings[i].tolist()}")


Some weights of BertModel were not initialized from the model checkpoint at retrained_bert_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Token: [CLS], Contextual Embedding: [-0.16548655927181244, 0.42242059111595154, -0.1730107218027115, -0.42913374304771423, -0.46040207147598267, 0.3970699608325958, 0.5751665234565735, 0.34414541721343994, 0.016702381893992424, -0.30061954259872437, 0.001547170802950859, 0.35652825236320496, -0.021018734201788902, 0.019437585026025772, -0.4229888916015625, 0.3138127028942108, 0.38839423656463623, 0.33990782499313354, -0.17416682839393616, -0.24316447973251343, 0.3460802137851715, 0.11626305431127548, 0.3377280831336975, 0.3501357436180115, 0.23820652067661285, -0.3388911783695221, 0.07611831277608871, -0.08890625834465027, 0.010880580171942711, 0.29468780755996704, 0.4664190709590912, 0.22173099219799042, 0.18172143399715424, -0.19887958467006683, 0.27284449338912964, -0.31337231397628784, 0.3334217071533203, -0.31042152643203735, 0.19250619411468506, -0.5054426789283752, -0.4078198969364166, 0.2948157489299774, 0.2050279825925827, 0.2442426085472107, 0.04436030983924866, 0.15232050418

In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate masked input sequences for BERT Masked Language Modeling (MLM)
mlm_input_texts = ["[CLS] " + " ".join(tokens) + " [SEP]" for tokens in tokenized_texts]
mlm_input_ids = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text)) for text in mlm_input_texts]

In [ ]:
from transformers import BertForMaskedLM, BertConfig

# Path to the folder containing the retrained model files
model_folder_path = "retrained_bert_model/"

# Load the configuration
config = BertConfig.from_pretrained(model_folder_path + "config.json")

# Load the model
model = BertForMaskedLM.from_pretrained(model_folder_path, config=config)

# Optionally, move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Ensure the model is in evaluation mode
embeddings = []
model.eval()

with torch.no_grad():
    for input_ids in mlm_input_ids:
        input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension and move to GPU
        outputs = model(input_ids=input_ids)
        hidden_states = outputs[0]  # Get the output tensor directly
        embeddings.append(hidden_states.mean(dim=1).squeeze().cpu().numpy())  # Use the mean of the hidden states and move back to CPU


In [ ]:
import torch

# Path to the pre-trained model file
model_path = "bert_checkpoint_epoch_2.pt"

# Load the model
model = torch.load(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Assuming mlm_input_ids is a list of input IDs

with torch.no_grad():
    embeddings = []
    for input_ids in mlm_input_ids:
        input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)  # Add batch dimension and move to GPU
        outputs = model(input_ids=input_ids)
        hidden_states = outputs[0]  # Get the output tensor directly
        embeddings.append(hidden_states.mean(dim=1).squeeze().cpu().numpy())  # Use the mean of the hidden states and move back to CPU


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

#LDA Implementation

In [ ]:
import pandas as pd

# Read the first 30,000 rows of the CSV file
df = pd.read_csv('training.csv', header=None, names=['id', 'timestamp', 'date', 'query', 'user', 'text'], nrows=30000)

# Extract the text from the last column
text_data = df['text']

# Display the first few rows of the text data
print(text_data.head())

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object


In [ ]:
# Apply Latent Dirichlet Allocation (LDA) for topic modeling
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = vectorizer.fit_transform([" ".join(tokens) for tokens in tokenized_texts])
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=5, random_state=42)

In [ ]:
# Print the top words for each topic
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic {topic_idx + 1}:")
    top_words_idx = topic.argsort()[:-10 - 1:-1]
    feature_names = vectorizer.get_feature_names_out()
    top_words = [feature_names[i] for i in top_words_idx]
    print(top_words)

Topic 1:
['central', 'high', 'task', 'quite', 'tend', 'admit', 'west', 'like', 'project', 'push']
Topic 2:
['draw', 'prepare', 'store', 'financial', 'career', 'score', 'value', 'center', 'family', 'collection']
Topic 3:
['boy', 'performance', 'political', 'color', 'edge', 'goal', 'result', 'low', 'court', 'test']
Topic 4:
['american', 'remember', 'teacher', 'modern', 'rate', 'play', 'friend', 'manage', 'allow', 'world']
Topic 5:
['soldier', 'let', 'evening', 'light', 'add', 'forget', 'machine', 'police', 'fall', 'audience']


#BERT Performance

In [ ]:
import pandas as pd

# Read the first 30,000 rows of the CSV file
df = pd.read_csv('training.csv', header=None, names=['id', 'timestamp', 'date', 'query', 'user', 'text'], nrows=30000)

# Extract the text from the last column
text_data = df['text']

# Display the first few rows of the text data
print(text_data.head())


0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('retrained_bert_model/')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the text data
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in text_data]

# Generate embeddings for each tokenized text
embeddings = []
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
with torch.no_grad():
    for input_ids in tokenized_texts:
        input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
        outputs = model(input_ids=input_ids)
        hidden_states = outputs[0]
        embeddings.append(hidden_states.mean(dim=1).squeeze().cpu().numpy())


In [ ]:
print(embeddings[10])

[-12.816586 -12.927038 -12.921815 ... -12.048332 -12.03392  -12.870981]


In [ ]:
batch_size = 32
num_batches = len(tokenized_texts) // batch_size + 1

def calculate_similarity_batched(embeddings):
    sim_sum = 0.0
    pairs = 0
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(tokenized_texts))
        batch_embeddings = embeddings[start_idx:end_idx]
        similarities = cosine_similarity(batch_embeddings)
        for j in range(len(batch_embeddings)):
            for k in range(j+1, len(batch_embeddings)):
                try:
                  sim_sum += similarities[i][j]
                  pairs += 1
                except IndexError:
                  continue
    return sim_sum / pairs if pairs > 0 else 0

similarity = calculate_similarity_batched(embeddings)
print(f"Average Semantic Similarity for Fine-Tuned BERT (Batched): {similarity}")


Average Semantic Similarity for Fine-Tuned BERT (Batched): 0.9978182296899538


#Experiments

In [ ]:
from transformers import BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from collections import defaultdict
import numpy as np

In [ ]:
batch_size = 32
num_batches = len(tokenized_texts) // batch_size + 1

def calculate_similarity(embeddings):
    try:
        if isinstance(embeddings[0], np.ndarray):
            # Assuming BERT-style embeddings
            embeddings = np.array(embeddings)
            return np.mean(cosine_similarity(embeddings))
        elif isinstance(embeddings[0], list):
            # Assuming LDA-style embeddings
            flattened_embeddings = [emb for topic in embeddings for emb in topic]
            flattened_embeddings = np.array(flattened_embeddings)
            return np.mean(cosine_similarity(flattened_embeddings))
    except IndexError:
        return 0.0

In [ ]:
def cluster_embeddings(embeddings, n_clusters=3):
    clusters = defaultdict(list)
    try:
        if isinstance(embeddings[0], np.ndarray):
            # Assuming BERT-style embeddings
            embeddings = np.array(embeddings)
            kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings)
            for i, label in enumerate(kmeans.labels_):
                clusters[label].append(i)
        elif isinstance(embeddings[0], list):
            # Assuming LDA-style embeddings
            for i, topic in enumerate(embeddings):
                for j, emb in enumerate(topic):
                    clusters[j].append(i)
    except IndexError:
        pass

    return clusters


In [ ]:
# Calculate semantic similarity for BERT embeddings
similarity = calculate_similarity_batched(embeddings)
print(f"Average Semantic Similarity for Fine-Tuned BERT (Batched): {similarity}")

# Cluster words for BERT embeddings
n_clusters_bert = 3  # Number of clusters/topics
clusters_bert = cluster_words(embeddings, n_clusters_bert)
for cluster_id, text_indices in clusters_bert.items():
    texts_in_cluster = [df.iloc[i]['Text'] for i in text_indices]
    print(f"Cluster {cluster_id} (BERT): {texts_in_cluster}")

# Sample LDA embeddings (replace with actual embeddings)
lda_embeddings = np.random.rand(5, 10)  # Assuming 5 topics and 10 words per topic

# Calculate semantic similarity for LDA embeddings
similarity_lda = calculate_similarity(lda_embeddings)
print(f"Average Semantic Similarity for LDA: {similarity_lda}")

# Cluster words for LDA embeddings
n_clusters_lda = 3  # Number of clusters/topics
clusters_lda = cluster_words(lda_embeddings, n_clusters_lda)
for cluster_id, word_indices in clusters_lda.items():
    words_in_cluster = [top_words[i] for i in word_indices]  # Assuming top_words is defined
    print(f"Cluster {cluster_id} (LDA): {words_in_cluster}")


Average Semantic Similarity for Fine-Tuned BERT (Batched): 0.9978182296899538


In [ ]:
print(f"Number of embeddings: {len(embeddings)}")


Number of embeddings: 30000


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd

# Assuming 'df' is your DataFrame containing the text data
text_data = df['Text'].tolist()

# Tokenize the text data
tokenized_data = [word_tokenize(text.lower()) for text in text_data]

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained model
model.save("word2vec.model")


In [ ]:
from gensim.models import Word2Vec

# Load Word2Vec model
word2vec_model = Word2Vec.load("word2vec.model")

# Get the embedding for a word
lda_embedding = word2vec_model.wv["word"]


In [ ]:
print(lda_embedding)

[-1.53681874e-01  3.34077954e-01  6.13060184e-02  1.15056045e-01
  4.58134599e-02 -2.03509197e-01 -5.28144091e-03  3.16840082e-01
 -1.17110148e-01 -1.07344635e-01 -1.85822129e-01 -5.34025505e-02
 -1.76383287e-01  5.62344268e-02  9.45780873e-02 -3.23079638e-02
 -2.92560998e-02 -1.35427758e-01  3.63722891e-02 -2.16844857e-01
  2.02140942e-01  1.39758036e-01  1.19431518e-01  3.56963314e-02
 -3.43460031e-02  5.01173362e-02 -1.29042655e-01 -1.14944458e-01
 -5.01004159e-02 -1.50970057e-01  4.43034843e-02  2.02390961e-02
 -5.55405468e-02 -2.66725905e-02  6.29808307e-02  1.42276762e-02
  3.51009928e-02 -2.70971656e-01 -1.20126240e-01 -2.93842733e-01
 -1.48476735e-01  9.02284309e-02  2.39088405e-02 -1.23320028e-01
 -9.07219499e-02 -6.23082593e-02 -1.85868114e-01  1.71486184e-01
  2.27756441e-01  1.94572672e-01 -4.10193615e-02 -1.85669139e-01
 -1.22118965e-01 -4.81776632e-02 -1.95708603e-01  7.83667862e-02
  3.19023430e-01 -2.09953234e-01 -1.55210003e-01  1.94393210e-02
  1.14385352e-01  2.25200

In [ ]:
import pandas as pd
from transformers import BertTokenizer
import random

# Assuming 'df' is your DataFrame with a column 'text'

# Select a random row index
random_index = random.randint(0, len(df) - 1)

# Get the text from the random row
text = df.loc[random_index, 'Text']

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
tokenized_text = tokenizer.tokenize(text)

print("Random text from dataset:", text)
print("Tokenized text:", tokenized_text)


Random text from dataset: Generation mother back summer. Too tree new.
Agreement north tax religious arrive democratic. Be test black something eat prepare parent. Religious both quite never purpose officer former.
Tokenized text: ['generation', 'mother', 'back', 'summer', '.', 'too', 'tree', 'new', '.', 'agreement', 'north', 'tax', 'religious', 'arrive', 'democratic', '.', 'be', 'test', 'black', 'something', 'eat', 'prepare', 'parent', '.', 'religious', 'both', 'quite', 'never', 'purpose', 'officer', 'former', '.']


In [ ]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
tokenized_texts = [tokenizer.tokenize(text) for text in df['Text']]

# Sample BERT embeddings (replace with actual embeddings)
bert_embeddings = np.random.rand(len(tokenized_texts), 768)

In [ ]:
# Sample LDA embeddings (replace with actual embeddings)
lda_embeddings = np.random.rand(5, 10)  # Assuming 5 topics and 10 words per topic

# Sample BERT embeddings (replace with actual embeddings)
bert_embeddings = np.random.rand(10, 768)  # Assuming 10 words and 768 dimensions

# Calculate semantic similarity for both LDA and BERT embeddings
lda_similarity = calculate_similarity(lda_embeddings)
bert_similarity = calculate_similarity(bert_embeddings)
print(f"Average Semantic Similarity for LDA: {lda_similarity}")
print(f"Average Semantic Similarity for BERT: {bert_similarity}")

# Cluster words for both LDA and BERT embeddings
n_clusters = 3  # Number of clusters/topics
lda_clusters = cluster_embeddings(lda_embeddings, n_clusters)
bert_clusters = cluster_embeddings(bert_embeddings, n_clusters)
print("LDA Clusters:")
print(lda_clusters)
print("\nBERT Clusters:")
print(bert_clusters)


Average Semantic Similarity for LDA: 0.802945583874326
Average Semantic Similarity for BERT: 0.7814621985024458
LDA Clusters:
defaultdict(<class 'list'>, {1: [0, 3], 2: [1, 2], 0: [4]})

BERT Clusters:
defaultdict(<class 'list'>, {0: [0, 3, 8], 2: [1, 2, 4, 7], 1: [5, 6, 9]})


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
